# Data (complete)

In [1]:
import numpy as np

data = np.load("./data/mars_for_students.npz")
training_set = data["training_set"]

X_train = training_set[:, 0]
y_train = training_set[:, 1]
X_test = data["test_set"]

print(X_train.shape)

(2615, 64, 128)


# 

# Model 

In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F

class conv_block(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(in_c, out_c, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_c),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_c, out_c, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_c),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)

class encoder_block(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()

        self.conv = conv_block(in_c, out_c)
        self.pool = nn.MaxPool2d((2, 2))

    def forward(self, x):
        s = self.conv(x)
        p = self.pool(s)
        return s, p

class attention_gate(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()

        self.Wg = nn.Sequential(
            nn.Conv2d(in_c[0], out_c, kernel_size=1, padding=0),
            nn.BatchNorm2d(out_c)
        )
        self.Ws = nn.Sequential(
            nn.Conv2d(in_c[1], out_c, kernel_size=1, padding=0),
            nn.BatchNorm2d(out_c)
        )
        self.relu = nn.ReLU(inplace=True)
        self.output = nn.Sequential(
            nn.Conv2d(out_c, out_c, kernel_size=1, padding=0),
            nn.Sigmoid()
        )

    def forward(self, g, s):
        Wg = self.Wg(g)
        Ws = self.Ws(s)
        out = self.relu(Wg + Ws)
        out = self.output(out)
        return out * s

class decoder_block(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()

        self.up = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
        self.ag = attention_gate(in_c, out_c)
        self.c1 = conv_block(in_c[0]+out_c, out_c)

    def forward(self, x, s):
        x = self.up(x)
        s = self.ag(x, s)
        x = torch.cat([x, s], axis=1)
        x = self.c1(x)
        return x

class attention_unet(nn.Module):
    def __init__(self,in_channels=1,out_channels=5,deep_sup=True):
        super().__init__()
        self.deep_sup=deep_sup

        self.e1 = encoder_block(in_channels, 64)
        self.e2 = encoder_block(64, 128)
        self.e3 = encoder_block(128, 256)
        self.e4 = encoder_block(256, 512)

        self.b1 = conv_block(512, 1024)
        
        self.d4 = decoder_block([1024, 512], 512)
        self.d3 = decoder_block([512, 256], 256)
        self.d2 = decoder_block([256, 128], 128)
        self.d1 = decoder_block([128, 64], 64)

        self.output = nn.Conv2d(64, out_channels, kernel_size=1, padding=0)
        self.o_d2=nn.Conv2d(128, out_channels, kernel_size=1, padding=0)
        self.o_d3=nn.Conv2d(256, out_channels, kernel_size=1, padding=0)
        self.o_d4=nn.Conv2d(512, out_channels, kernel_size=1, padding=0)

    def forward(self, x):
        s1, p1 = self.e1(x)
        s2, p2 = self.e2(p1)
        s3, p3 = self.e3(p2)
        s4, p4 = self.e4(p3)
        
        b1 = self.b1(p4)
        
        d4 = self.d4(b1, s4)
        d3 = self.d3(d4, s3)
        d2 = self.d2(d3, s2)
        d1 = self.d1(d2, s1)


        if self.deep_sup:
            d2=F.interpolate(self.o_d2(d2),scale_factor=2,mode="bilinear",align_corners=True)
            d3=F.interpolate(self.o_d3(d3),scale_factor=4,mode="bilinear",align_corners=True)
            d4=F.interpolate(self.o_d4(d4),scale_factor=8,mode="bilinear",align_corners=True)
            output = self.output(d1)
            return output,d2,d3,d4
        else : 
            output = self.output(d1)
            return output

In [ ]:
import torch
import torch.nn as nn

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, padding=padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size, padding=padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, in_channels, out_channels, large_kernels=False):
        super(UNet, self).__init__()
        kernel_size = 5 if large_kernels else 3
        padding = 2 if large_kernels else 1

        self.encoder1 = DoubleConv(in_channels, 64, kernel_size, padding)
        self.encoder2 = DoubleConv(64, 128, kernel_size, padding)
        self.encoder3 = DoubleConv(128, 256, kernel_size, padding)
        self.encoder4 = DoubleConv(256, 512, kernel_size, padding)

        self.pool = nn.MaxPool2d(2)

        self.bottleneck = DoubleConv(512, 1024, kernel_size, padding)

        self.upconv4 = nn.ConvTranspose2d(1024, 512, 2, stride=2)
        self.decoder4 = DoubleConv(1024, 512, kernel_size, padding)
        self.upconv3 = nn.ConvTranspose2d(512, 256, 2, stride=2)
        self.decoder3 = DoubleConv(512, 256, kernel_size, padding)
        self.upconv2 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.decoder2 = DoubleConv(256, 128, kernel_size, padding)
        self.upconv1 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.decoder1 = DoubleConv(128, 64, kernel_size, padding)

        self.output_layer = nn.Conv2d(64, out_channels, kernel_size=1)
        
        # Auxiliary outputs for deep supervision
        self.aux_output4 = nn.Conv2d(512, out_channels, kernel_size=1)
        self.aux_output3 = nn.Conv2d(256, out_channels, kernel_size=1)
        self.aux_output2 = nn.Conv2d(128, out_channels, kernel_size=1)
        
        # Upsampling layers for resizing auxiliary outputs
        self.upsample4 = nn.Upsample(scale_factor=8, mode='bilinear', align_corners=True)
        self.upsample3 = nn.Upsample(scale_factor=4, mode='bilinear', align_corners=True)
        self.upsample2 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

    def forward(self, x):
        e1 = self.encoder1(x)
        e2 = self.encoder2(self.pool(e1))
        e3 = self.encoder3(self.pool(e2))
        e4 = self.encoder4(self.pool(e3))

        b = self.bottleneck(self.pool(e4))

        d4 = self.upconv4(b)
        d4 = torch.cat((d4, e4), dim=1)
        d4 = self.decoder4(d4)
        aux4 = self.upsample4(self.aux_output4(d4))  # Resize auxiliary output

        d3 = self.upconv3(d4)
        d3 = torch.cat((d3, e3), dim=1)
        d3 = self.decoder3(d3)
        aux3 = self.upsample3(self.aux_output3(d3))  # Resize auxiliary output

        d2 = self.upconv2(d3)
        d2 = torch.cat((d2, e2), dim=1)
        d2 = self.decoder2(d2)
        aux2 = self.upsample2(self.aux_output2(d2))  # Resize auxiliary output

        d1 = self.upconv1(d2)
        d1 = torch.cat((d1, e1), dim=1)
        d1 = self.decoder1(d1)

        final_output = self.output_layer(d1)

        return final_output, aux4, aux3, aux2

class DoubleUNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleUNet, self).__init__()
        self.unet1 = UNet(in_channels, out_channels, large_kernels=True)
        self.unet2 = UNet(out_channels + in_channels, out_channels, large_kernels=False)

    def forward(self, x):
        # Forward pass through the first UNet
        out1, aux4_1, aux3_1, aux2_1 = self.unet1(x)
        normalized_out1 = (out1 - out1.mean(dim=(2, 3), keepdim=True)) / (out1.std(dim=(2, 3), keepdim=True) + 1e-8)
        combined_input = torch.cat((x, normalized_out1), dim=1)

        # Forward pass through the second UNet
        out2, aux4_2, aux3_2, aux2_2 = self.unet2(combined_input)

        return out2,aux2_1,aux4_2, aux3_2, aux2_2

# Loss

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DiceLoss(nn.Module):
    def __init__(self, smooth=1.0):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, preds, targets):
        preds = torch.softmax(preds, dim=1) 
        num_classes = preds.shape[1]
        targets_one_hot = F.one_hot(targets, num_classes=num_classes).permute(0, 3, 1, 2).float()

        dice_loss = 0.0
        for c in range(num_classes):

            pred_c = preds[:, c, :, :]
            target_c = targets_one_hot[:, c, :, :]

            # IoU
            intersection = torch.sum(pred_c * target_c, dim=(1, 2))  
            union = torch.sum(pred_c + target_c, dim=(1, 2)) 

            dice = (2.0 * intersection + self.smooth) / (union + self.smooth)
            dice_loss += (1 - dice).mean()  # Mean on the batch

        # mean on the classes
        return dice_loss / num_classes
    
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction  # 'mean', 'sum' or 'none'

    def forward(self, inputs, targets):
        inputs = torch.softmax(inputs, dim=1)  # Softmax
        targets = targets.long()

        N, C, H, W = inputs.size()
        inputs = inputs.view(N, C, -1)  # (batch_size, num_classes, height * width)
        targets = targets.view(N, -1)   # (batch_size, height * width)

        one_hot = torch.zeros(N, C, H * W).to(inputs.device)
        one_hot.scatter_(1, targets.unsqueeze(1), 1)  # Good shape

        p_t = (inputs * one_hot).sum(dim=1)  # Probability target predictions
        loss = -self.alpha * (1 - p_t) ** self.gamma * torch.log(p_t + 1e-8)

        # reduction
        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:
            return loss

class CombinedLoss(nn.Module):
    def __init__(self):
        super(CombinedLoss, self).__init__()
        self.dice_loss = DiceLoss()
        self.focal_loss = FocalLoss()

    def forward(self, preds, targets):
        dice = self.dice_loss(preds, targets)
        focal = self.focal_loss(preds, targets)
        return 0.5 * dice + 0.5 * focal

# Training

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset,random_split
import numpy as np
from CpNet import CPNet 
from attunet import attention_unet
import torchmetrics
import metrics
from u2net import U2NET

# Hyperparameter
batch_size = 32
learning_rate = 1e-4
epochs = 100
num_classes = 5
device = "cuda" if torch.cuda.is_available() else "cpu"


X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
print(X_train_tensor.shape,y_train_tensor.shape)


train_dataset = TensorDataset(X_train_tensor.to(device), y_train_tensor.to(device))


train_dataset, val_dataset = random_split(train_dataset, [0.95, 0.05])
print(len(train_dataset))
print(len(val_dataset))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


#model=attention_unet(in_channels=1)
model=DoubleUNet(in_channels=1,out_channels=5)
model = model.to(device)


criterion = CombinedLoss()


optimizer = optim.Adam(model.parameters(), lr=learning_rate)
jaccard = torchmetrics.JaccardIndex(task="multiclass", num_classes=5).to(device)


In [ ]:
def plot_output_and_label(output, label):

    output = output.squeeze(0).detach().cpu().numpy()  
    label = label.squeeze(0).detach().cpu().numpy() 


    output_class = np.argmax(output, axis=0)

 
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))


    ax[0].imshow(output_class, cmap='jet') 
    ax[0].set_title('Output (Predicted Classes)')
    ax[0].axis('off')

    ax[1].imshow(label, cmap='jet') 
    ax[1].set_title('Label (Ground Truth)')
    ax[1].axis('off')

    plt.show()

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
from torchmetrics.classification import JaccardIndex
import albumentations as A

# Configuration
train_val_split_ratio = 0.95 
patience = 10
min_delta = 0.001  
best_val_loss = float('inf')
early_stopping_counter = 0

# Optimizer et Scheduler
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.5)


def deep_supervision_loss(outputs, labels, criterion, weights=None):
    if weights is None:
        weights = [1.0 / len(outputs)] * len(outputs)

    total_loss = 0.0
    for i, output in enumerate(outputs):
        total_loss += weights[i] * criterion(output, labels)

    return total_loss


# Training loop
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    running_miou = 0.0

    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Loss calculation
        loss=deep_supervision_loss(outputs,labels,criterion)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_miou += jaccard(outputs[0], labels)

    epoch_loss = running_loss / len(train_loader)
    epoch_miou = running_miou / len(train_loader)

    # Visualisation
    if epoch % 10 == 0:
        plot_output_and_label(outputs[0][0], labels[0])

    # Validation
    model.eval()
    val_loss = 0.0
    val_miou = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            labels = labels.squeeze(1)
            loss = deep_supervision_loss(outputs,labels,criterion)
            val_loss += loss.item()
            val_miou += jaccard(outputs[0], labels)
    val_loss /= len(val_loader)
    val_miou /= len(val_loader)

    # Early stopping
    if val_loss < best_val_loss - min_delta:
        best_val_loss = val_loss
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1

    if early_stopping_counter >= patience:
        print(f"Arrêt anticipé à l'époque {epoch + 1}")
        break

    
    lr_scheduler.step(val_loss)

    # metrics
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss:.4f}, mIoU: {epoch_miou:.4f}, -Val_Loss: {val_loss:.4f}, Val_mIoU: {val_miou:.4f} Lr: {optimizer.param_groups[0]['lr']}")

# Submission

In [ ]:
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
test_dataset = TensorDataset(X_test_tensor.to(device))
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
all_preds=[]
with torch.no_grad():
    for batch  in test_loader:
        preds = model(batch[0])
        all_preds.append(preds[0].cpu().numpy())
all_preds = np.vstack(all_preds)
all_preds = np.argmax(all_preds, axis=1)
print(f"Predictions shape: {all_preds.shape}")

In [ ]:
import pandas as pd 
def y_to_df(y) -> pd.DataFrame:
    """Converts segmentation predictions into a DataFrame format for Kaggle."""
    n_samples = len(y)
    y_flat = y.reshape(n_samples, -1)
    df = pd.DataFrame(y_flat)
    df["id"] = np.arange(n_samples)
    cols = ["id"] + [col for col in df.columns if col != "id"]
    return df[cols]

# Create and download the csv submission file
submission_filename = f"./submission/submission_dp1.csv"
submission_df = y_to_df(all_preds)
submission_df.to_csv(submission_filename, index=False)